In [ ]:
'''Remember put <GO> in front of the inferenced inputs'''
a = 300
X_test = X_train[a:a+16]
for i in range(0, len(X_test)):
#     X_test[i] = np.concatenate([tokens_embedding[0], X_test[i]])
    X_test[i] = [tokens_id[0]] + X_test[i]
    X_test[i] = X_test[i][0:3]

checkpoint = '../../model/' + 'enc_dec_layer_2_dim_512_pre_emb_None_beam_search.ckpt-' + '1160'

loaded_graph = tf.Graph()
with tf.Session(graph = loaded_graph) as sess:
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)
    
    '''show trainable variables'''
#     temp_show = [n.name for n in tf.get_default_graph().as_graph_def().node]
#     for o in temp_show:
#         print(o)

    e_i = loaded_graph.get_tensor_by_name('encoder_inputs:0')
#     train_logits = loaded_graph.get_tensor_by_name('Decoder/train_predictions:0')
    inf_logits = loaded_graph.get_tensor_by_name('inference_predictions:0')
    e_i_l = loaded_graph.get_tensor_by_name('encoder_inputs_length:0')
    
    X_test_batch, X_test_batch_length = Touka.prepare_batch(X_test, 
                                                            dim = 2,
#                                                             emb_size = 300, 
                                                            time_major = False)
    
#     answer_logits1 = sess.run(train_logits, {e_i: X_test_batch, e_i_l: X_test_batch_length})
    answer_logits2 = sess.run(inf_logits, {e_i: X_test_batch, e_i_l: X_test_batch_length})